In [6]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [59]:
#검색할 식당 리스트 입력
items = ['순도리', '로흐', '버터핑거팬케익스', '파스타집이야']

In [60]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(items) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    #카카오맵 접속 후 검색창에 입력
    driver.get("https://map.kakao.com/")
    
    #검색창에 식당명 입력   
    searchbox = driver.find_element(by='xpath', value="//input[@id='search.keyword.query']") 
    searchbox.send_keys(item)
    time.sleep(2)
    
  
    #검색 버튼 누르기
    searchbutton = driver.find_element(by='xpath', value="//button[@id='search.keyword.submit']") 
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    #검색한 식당이 있을 때 리뷰 정보 가져오기
    if len(driver.find_elements(by='xpath', value="//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        
        #리뷰수 버튼 클릭
        reviewbutton = driver.find_element(by='xpath', value="//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href")
        
        #리뷰창이 새 탭에서 뜨도록 처리 
        tabs = driver.window_handles
        driver.switch_to.window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(2)
        
        

        #스크롤 내리기
        for c in range(0,1):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        
        
        #리뷰 데이터 스크래핑을 위한 html 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        
        #리뷰 수가 0이 아닌 경우 리뷰글 가져오기
        if len(review_lists) != 0 :
            
            #후기 더보기가 있으면 모두 눌러주기 
            while True: 
                try:
                    if driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more").textContent') == "후기 더보기":
                        driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.ico_comm.ico_more").click()')
                        time.sleep(2) 
                        
                except:
                    print("더보기 종료")
                    break 
                
            #후기 더보기를 모두 누른 후 리뷰 리스트 재정의
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            review_lists = soup.select('.list_evaluation > li')
            print(len(review_lists))
                    
            
            #리뷰 정보가 있는 경우 리뷰글, 식당 평균 평점, 리뷰 작성 시간 가져오기     
            try:
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.comment_info > p > span') # 리뷰
                        rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text #식당평균평점
                        timestamp = review.select(' div > span.time_write') #시간정보
                        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            item,
                            rate,
                            user_review[0].text,
                            timestamp[0].text
                        ]
                        )            
    
                    time.sleep(1)
                
               
            except:
                print("더 이상 리뷰가 존재하지 않습니다.")
                
                
                
        else :
            #리뷰 정보가 없는 경우 식당명과 평균 평점만 데이터 프레임에 추가
            rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text
            rev_list.append(
                [
                    item,
                    rate,
                ]
            ) 

                    
            print("리뷰가 없습니다")
        
        #검색한 창 닫고 카카오맵 검색 페이지로 돌아가기    
        driver.close()
        driver.switch_to.window(tabs[0])
        print("기본 페이지로 돌아가기")
            
    else:
        print("식당이 존재하지 않음")
        
driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기    
columns = ["name", "rate", "review", "timestamp"]
df = pd.DataFrame(rev_list, columns=columns)
df    

진행상황 : 1/4
식당 존재
더보기 종료
22
기본 페이지로 돌아가기
진행상황 : 2/4
식당 존재
더보기 종료
18
기본 페이지로 돌아가기
진행상황 : 3/4
식당 존재
더보기 종료
48
기본 페이지로 돌아가기
진행상황 : 4/4
식당 존재
리뷰가 없습니다
기본 페이지로 돌아가기


,name,rate,review,timestamp
0,순도리,2.4,순대국에 소주 먹으러 갔는데 소주가 7천원..,2022.10.07.
1,순도리,2.4,서빙이 -1별점 하는거 같음내가 여길 두번을 갔네 …쩝,2022.09.26.
2,순도리,2.4,볶음밥에 밥이 너무 없어서 아쉬운거 빼구 다 좋아요~~~,2022.09.23.
3,순도리,2.4,서빙보는 50대 짱깨 아줌마환타음료 시켰는데 테이블에 노룩패스로 주더라ㅋㅋㅋㅋ일부러...,2022.06.23.
4,순도리,2.4,맛은 무난한데 직원들 불친절함 걍 벙어리임,2022.06.19.
...,...,...,...,...
84,버터핑거팬케익스,3.3,별점 기준1) 재방문 의사 없음.2) 찾아서 갈만큼은 아님. 주로 가성비 떨어짐.3...,2016.04.21.
85,버터핑거팬케익스,3.3,,2016.04.20.
86,버터핑거팬케익스,3.3,,2016.03.23.
87,버터핑거팬케익스,3.3,맛:★★★분위기:★★★기타: 팬케이크는 맛있는데 전반적으로 달다. 양도 은근히 많...,2016.03.11.


In [61]:
#데이터 프레임 전처리#

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

#중복값 제거
#df = df.drop_duplicates(['review'], keep='first',ignore_index = True)

#리뷰글이 없는 결측치 제거
#df['review']=df['review'].dropna()

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

버터핑거팬케익스    48
순도리         22
로흐          18
파스타집이야       1
Name: name, dtype: int64


In [62]:
#스크래핑한 데이터 프레임을 csv로 저장
df.to_csv('sample.csv', encoding='utf-8-sig')